## Crossmatch TAP query demo 
This notebook was written to show how to access the crossmatch results for a known list of sourceIDs. Initially a sample list of IDs was 
obtained by querying the table itself. Andy Wilson then provided a file containing the actual IDs they were interested in so the notebook was modified to read 
in that file.

import and set-up

In [1]:
# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

service = get_tap_service("tap")
assert service is not None
assert service.baseurl == "https://rsp.lsst.ac.uk/api/tap"

# see what databases there are
query = "SELECT * FROM tap_schema.schemas"
resultsSchema = service.search(query).to_table()
resultsSchema

description,schema_index,schema_name,utype
str512,int32,str64,str512
no database description provided,1,dc2_run2_1i_dr1b,
gaia_source catwise_2020 matches from Edinburgh,2,gaiaxcatwise,
allsky gaia_source catwise_2020 matches from Edinburgh,3,gaiaxcatwiseallsky,
A TAP-standard-mandated schema to describe tablesets in a TAP 1.1 service,100000,tap_schema,
no database description provided,0,UDS,
UWS Metadata,120000,uws,
VIDEO/HSC database from WP3.5,4,video,
VIKING/HSC database from WP3.5,5,viking,


In [2]:
import pandas
# have a look at the columns
query = "SELECT * from TAP_SCHEMA.columns "+\
                 "WHERE table_name = 'gaiaxcatwise.matches_source'"
res = service.search(query)
print(res.fieldnames)
results_table = res.to_table().to_pandas()
print(results_table[['column_name','datatype']])

('"size"', 'arraysize', 'column_index', 'column_name', 'datatype', 'description', 'indexed', 'principal', 'std', 'table_name', 'ucd', 'unit', 'utype', 'xtype')
             column_name datatype
0               ab_flags     char
1             ag_gspphot    float
2       ag_gspphot_lower    float
3       ag_gspphot_upper    float
4    astrometric_chi2_al    float
..                   ...      ...
363         wise_fit_sig   double
364              wise_ra   double
365                   wx    float
366                   wy    float
367                   xi   double

[368 rows x 2 columns]


Simple count query, takes some minutes to complete

In [3]:
# simple count rows query, answer is 104097233
query = "select count(*) as c from gaiaxcatwise.matches_source"
results = service.search(query)

print(results)

<Table length=1>
    c       h   
  int64   object
--------- ------
104097233      1


In [4]:
print(type(results))
table=results.to_table()
print(type(table))

<class 'pyvo.dal.tap.TAPResults'>
<class 'astropy.table.table.Table'>


In [5]:
# test to get some sourceIds
query="select source_id from gaiaxcatwise.matches_source as x  limit 1000000"
results = service.search(query)
table = results.to_table()
print(table)

     source_id     
-------------------
4263950244784476160
4263950244784476288
4263950244784513536
4263950244784513664
4263950244784544896
4263950249076799744
4263950249076842624
4263950249076872832
4263950249081673088
4263950313504064512
                ...
2081496943002017024
2081496973058979712
2081496973058988800
2081496973059001728
2081496973059010688
2081496973059021312
2081496973059038720
2081496977357110656
2081496977357110784
2081496977357123712
2081496977357123840
Length = 1000000 rows


In [6]:
sourceIds=table['source_id'].data
print(sourceIds[0],max(sourceIds),min(sourceIds))

4263950244784476160 4503645227078649088 160988740193758592


load up the provided IDs

In [7]:
# load Andy's sourceIds
from astropy.io import fits
hdul = fits.open('GaiaEDR3_SourceIds_NGPv4.fits')
data = hdul[1].data # assuming the first extension is a table
hdul.close()
#print(data[0])
andysSourceIds=data['source_id']
print(andysSourceIds,len(andysSourceIds))

[4282338859504210048 4282338889562080256 4282338893863946624 ...
 2018615353311150592 2018615357602731008 2018615357625887488] 20039800


some 20 million IDs so split into 2001 chunks, and around 1000 Ids per chunk. Break after 2 loops as just an example


In [8]:
import numpy
# split up the sourceIds into N chunks (the number is the number of chunks not size of chunks)
chunks = numpy.array_split(numpy.array(andysSourceIds),2001) 
import datetime
i=0
from astropy.table import QTable, Table, Column, vstack
fullResults=Table()
#loop through chunks and submit query for those sourceIds, bringing back a few columns
# stacks all results into one table (but maybe they should be kept separate as this might be inefficient)
for chunk in chunks:
    print(i,len(chunk),datetime.datetime.now())
    
    inClause=','.join(str(sourceId) for sourceId in chunk) #numpy.array2string(chunk, separator=",")
    #print(inClause)
    query="select ra,dec,phot_bp_mean_mag,phot_g_mean_mag,phot_rp_mean_mag from gaiaxcatwise.matches_source where source_id in ("+inClause+")"
    #print(query)

    results = service.search(query)
    table = results.to_table()
    if i==0:
        fullResults=table
    else:
        fullResults=(vstack([fullResults, table]))
    #print(table)
    i+=1
    # set to low number for tests
    if i==2:
        break
print(len(fullResults))

0 10015 2023-11-20 14:50:07.854296
1 10015 2023-11-20 14:50:27.788841
11705


In [9]:
print(fullResults)

        ra                dec        ... phot_g_mean_mag phot_rp_mean_mag
------------------ ----------------- ... --------------- ----------------
282.79641546539625 5.652166847251163 ...         14.8337          13.8723
282.79712793588516 5.664816815400225 ...         19.4495          18.2937
282.80214227926376 5.655347556803341 ...         15.0963          14.2137
  282.793927141799 5.657256045332002 ...         18.3959          17.3438
282.79856528560197 5.663225003812569 ...         19.0944          17.9939
282.78183773120344 5.672781063624429 ...         15.2255           14.345
 282.8099464029409  5.66342591475745 ...         18.9705          17.7328
 282.8052750736308 5.665864747968357 ...         18.0998          17.0404
 282.8225793640582 5.685144834121797 ...         19.1917          18.0392
282.80466373966846 5.675024798024031 ...         18.7467          17.6633
               ...               ... ...             ...              ...
 284.2633592808782 6.640518390218935 .

In [ ]:
#import pyvo
#tap = pyvo.dal.TAPService('https://rsp.lsst.ac.uk/api/tap')
#tap.run_sync('select count(*) from SXDS.director')